In [1]:
%matplotlib inline
''' Python Tools '''
import numpy as np
np.set_printoptions(suppress=True) # Changes print functionality
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

'''PyTorch Functionality'''
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

'''Landscape Class'''
from Protein_Landscape.landscape_class import Protein_Landscape

'''Dataset Management'''
Datasets = 'Data/'


In [2]:
test = Protein_Landscape(csv_path=Datasets+"Small_NK.csv",gen_graph=True)


Building Protein Graph for entire dataset


  0%|          | 0/1000 [00:00<?, ?it/s]

True


100%|██████████| 1000/1000 [00:00<00:00, 57280.45it/s]

Calculating the number of extrema

        Protein Landscape class
            Number of Sequences : 1000
            Max Distance        : 3
            Number of Distances : 3
            Seed Sequence       : AAA
                Modified positions are shown in green
            Number of minima : 1
            Number of maxima : 1
            Normalized Extrema Ruggedness : 0.002
            R/S Ruggedness : 17.319433978287012
        


In [3]:
test.calc_neighbours(seq='ACL',explicit_neighbours=False)

False


AxisError: axis 1 is out of bounds for array of dimension 0

In [17]:
isinstance(3, np.integer)

False